### Manejo de archivos (II) y Estructuras

Una forma común de leer un archivo línea a línea es usando la función `fgets`. Su llamada es:
```
fgets(line, maxline, fp);
```
donde `line` es un arreglo de caracteres, `maxline` un entero y `fp` un puntero al archivo. La función guarda en el arreglo `line` la siguiente línea del archivo (incluyendo el caracter nueva línea `'\n'`). Si la línea es muy larga, la cantidad máxima de caracteres leidos es `maxline-1`. Asegurese de que el arreglo `line` tenga el tamaño suficiente. La función retorna `NULL` en caso que alcancemos el final del archivo o haya un error.

En el siguiente ejemplo las líneas del archivo tienen la forma:
```
(string solo letras y espacios),int
````
El valor int puede estar ausente. Para almacenar líneas usaremos un struct `ejemplo` con dos campos que podemos definir como sigue:
```
typedef struct n {
  char palabra[100];
  int x;
} ejemplo;
```

Acá el nombre de la estructura es `struct n`. Recuerde que el comando `typedef` nos permite renombrar tipos de datos. En este caso renombramos el tipo `struct n` a `ejemplo`.  

Asumiremos que el entero `x` es mayor o igual a `0`. Usaremos el valor `-1` para indicar que `x` no está definido.

En el siguiente ejemplo, leemos un archivo línea a línea y rellenamos un arreglo donde cada entrada contiene la estructura asociada a cada línea. Asumimos un máximo en la cantidad de líneas y un máximo en el tamaño de cada línea.


In [ ]:
%%writefile archivo.txt
Enrique Dieguez,3
Lino Montoya,
Manuel Villagra,23
Paulina Gaete,
Pablo Barcelo,101
Leticia Espinosa,93
Carolina Olivares,



In [ ]:
%%writefile test.c

#include <stdio.h>
#include <string.h>
#include <stdlib.h>

#define MAX_LINES 100
#define MAX_LINE_SIZE 100

typedef struct n {
  char palabra[100];
  int x;
} ejemplo;

int main(int argc, char *argv[])
{
    ejemplo arreglo[MAX_LINES];
    int i = 0;
    FILE *fp = fopen(argv[1], "r");
    char line[MAX_LINE_SIZE];
    char *token;
    // Leemos linea a linea 
    while(fgets(line, MAX_LINE_SIZE, fp) != NULL) // el arreglo line se rellena con los caracteres de la linea
    {
        // strtok(line, ",") entrega el substring desde el comienzo hasta la primera aparicion del delimitador "," (excluyendolo)
        // Copiamos el substring al campo palabra de la estructura
        strcpy(arreglo[i].palabra, strtok(line, ","));
        // Obtenemos el siguiente substring
        token = strtok(NULL, ",");
        if(strcmp(token, "\n") == 0) // caso que no hay int
          arreglo[i].x = -1;
        else
          arreglo[i].x = atoi(token); // atoi convierte un string a int
     
        i++;
    }
 
    // Mostramos el arreglo generado
    // En este punto la variable i contiene la cantidad de elementos en el arreglo
    for(int j=0; j < i; j++)
        printf("Entrada %d:\npalabra: %s\nx: %d\n", j, arreglo[j].palabra, arreglo[j].x);

    return 0;

}

In [ ]:
%%shell

gcc test.c -o test
./test archivo.txt

Ojo que las distintas llamadas de `strtok` están relacionadas. La primera siempre tiene la forma `strtok(str, del)`, donde `str` es el string a ser particionado y `del` es un string que contiene los delimitadores (pueden ser más de uno). Esta primera llamada nos entrega el primer "token", es decir, el substring desde el comienzo hasta el primer delimitador (excluyéndolo). Para obtener los siguientes token usamos las llamadas `strtok(NULL, del)`. 

Un detalle: en este caso, es importante que la última línea (en este caso "Carolina Olivares,") termine con un salto de línea. De lo contrario, la línea de código

```
token = strtok(NULL, ",");
```

arroja un `segmentation fault`. Esto tipo de error sucede cuando se quiere acceder una espacio de memoria prohibido. ¿Por qué sucede esto?
